In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from itertools import islice
from pandarallel import pandarallel

# 初始化 pandarallel
pandarallel.initialize()

def get_od_path(edge, node, station2node_dict, gdtollnode, beta=1.0):
    """
    获取OD路径和长度信息的函数，并计算每条路径的选择概率。

    输入参数：
    - edge: 包含路段信息的DataFrame，包括 'u'（起点节点ID）、'v'（终点节点ID）、'length'（路段长度）、'edge_id'（路段ID）等列。
    - node: 包含节点信息的DataFrame，至少包括 'id'（节点ID）列。
    - station2node_dict: 字典，将站点ID映射到节点ID的关系。
    - gdtollnode: 包含站点信息的DataFrame，至少包括 'id' 列。
    - beta: 模型参数，用于计算路径选择概率。默认值为1.0。

    输出结果：
    - None：路径和长度信息直接保存到文件中。
    """

    # 获取最短路径的函数（已优化，使用缓存）
    path_cache = {}  # 缓存已计算的最短路径

    def get_shortest_paths(start_station, end_station):
        if (start_station, end_station) not in path_cache:
            start_node = station2node_dict[start_station]
            end_node = station2node_dict[end_station]
            # 使用 shortest_simple_paths 获取最短的三条路径
            paths_generator = nx.shortest_simple_paths(G, source=start_node, target=end_node, weight='weight')
            shortest_paths = list(islice(paths_generator, 3))  # 获取前三条路径
            path_cache[(start_station, end_station)] = [list(map(int, path)) for path in shortest_paths]
            
        return path_cache[(start_station, end_station)]

    def save_paths_and_lengths(station_id_x, station_id_y, paths, beta):
        all_paths_details = []
        for path_id, path in enumerate(paths):
            path_details = [{'u': int(u), 'v': int(v), 'id': i,
                            'station_id_x': station_id_x, 'station_id_y': station_id_y,
                            'path_id': path_id}  # 添加路径标识符
                            for i, (u, v) in enumerate(zip(path[:-1], path[1:]))]
            all_paths_details.extend(path_details)
        
        od1_tmp = pd.DataFrame(all_paths_details)
        
        # 合并边的信息
        od1_tmp = pd.merge(od1_tmp, edge[['u', 'v', 'length', 'edge_id']], on=['u', 'v'])

        # 转换为整型并排序
        od1_tmp['length'] = od1_tmp['length'].astype(int)
        od1_tmp = od1_tmp.sort_values(by=['station_id_x', 'station_id_y', 'id'])

        # 计算累计长度
        od1_tmp['cumsumlength'] = od1_tmp.groupby(['station_id_x', 'station_id_y', 'path_id'])['length'].cumsum()

        od_dis_table = od1_tmp[['station_id_x', 'station_id_y', 'edge_id', 'cumsumlength', 'path_id']].sort_values(by = ['station_id_x', 'station_id_y', 'path_id'])
        
        # 计算每个OD的路径总长度
        od_length = od1_tmp.groupby(['station_id_x', 'station_id_y', 'path_id'])['cumsumlength'].max().reset_index()
        od_length.rename(columns={'cumsumlength': 'length'}, inplace=True)

        # 计算每条路径的选择概率
        od_length['exp_neg_beta_length'] = np.exp(-beta * od_length['length'])
        od_length['sum_exp_neg_beta_length'] = od_length.groupby(['station_id_x', 'station_id_y'])['exp_neg_beta_length'].transform('sum')
        od_length['probability'] = od_length['exp_neg_beta_length'] / od_length['sum_exp_neg_beta_length']
        
        # 准备合并所需的概率信息
        probabilities = od_length[['station_id_x', 'station_id_y', 'path_id', 'probability']]
        
        # 将选择概率合并到od_dis_table中
        od_dis_table_updated = pd.merge(
            od_dis_table,
            probabilities,
            on=['station_id_x', 'station_id_y', 'path_id'],
            how='left'
        )

        # 保存到CSV文件
        od_dis_table_updated.to_csv(f'中间数据/network/od_dis_tables/od_dis_table_{station_id_x}_{station_id_y}.csv', index=False)
        od_length.to_csv(f'中间数据/network/od_lengths/od_length_{station_id_x}_{station_id_y}.csv', index=False)

    G_edges = edge[['u','v','length']].values
    G_nodes = list(node['id'])

    # 先创建一个有向图
    G = nx.DiGraph()
    # 添加节点
    G.add_nodes_from(G_nodes) 
    # 添加边
    G.add_weighted_edges_from(G_edges)

    # 创建OD表
    o = gdtollnode[['id']]
    o.columns = ['station_id']
    o['flag'] = 1
    d = o.copy()
    od = pd.merge(o, d, on='flag')[['station_id_x', 'station_id_y']]
    # 随机取1000个OD用于测试
    od = od.sample(n=1000, replace=True) 
    
    print('获取OD的出行路径')
    for idx, row in od.iterrows():
        station_id_x = row['station_id_x']
        station_id_y = row['station_id_y']
        paths = get_shortest_paths(station_id_x, station_id_y)
        save_paths_and_lengths(station_id_x, station_id_y, paths, beta)

    return None


In [ ]:
import os
import pandas as pd

def process_and_merge_od_dis_tables(input_directory):
    """
    依次读入每个 od_dis_table 文件，利用 od_merge_table_reconstruct 函数生成对应 OD 的 new_od_dis，并将所有 new_od_dis 合并成一个文件。

    参数:
    - input_directory: 存储单独的 od_dis_table 文件的目录。
    - output_path: 合并后的 new_od_dis.csv 文件的输出路径。
    """
    # 初始化一个空的 DataFrame
    combined_new_od_dis = pd.DataFrame()

    # 遍历目录中的所有 od_dis_table 文件
    for filename in os.listdir(input_directory):
        if filename.startswith('od_dis_table_') and filename.endswith('.csv'):
            file_path = os.path.join(input_directory, filename)
            # 读取 od_dis_table 文件
            od_dis_table = pd.read_csv(file_path)
            # 利用 od_merge_table_reconstruct 生成对应 OD 的 new_od_dis
            new_od_dis = highway_model.od_merge_table_reconstruct(od_dis_table)
            # 将生成的 new_od_dis 合并到 combined_new_od_dis 中
            combined_new_od_dis = pd.concat([combined_new_od_dis, new_od_dis], ignore_index=True)

    # 保存合并后的 new_od_dis.csv 文件
    return combined_new_od_dis

# 示例调用
input_directory = 'od_dis_tables/'  # 存储单独的 od_dis_table 文件的目录
combined_new_od_dis = process_and_merge_od_dis_tables(input_directory)
